<h2> 1. Data Modelling using Linear Regression Model

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn import metrics

In [2]:
pd.set_option('display.max_columns', None)

In [3]:
df = pd.read_csv('./Clean EDA Exported files/df_dummies.csv')
test_df = pd.read_csv('./Clean EDA Exported files/test_df_dummies.csv')

Reading in the train (df_dummies.csv) and test (test_df_dummies.csv) files which we cleaned in our data cleaning python notebook.

In [4]:
df

,id,tranc_yearmonth,town,flat_type,flat_model,lease_commence_date,resale_price,tranc_year,tranc_month,lower,upper,mid,full_flat_type,floor_area_sqft,price_per_sqft,max_floor_lvl,year_completed,residential,commercial,market_hawker,multistorey_carpark,precinct_pavilion,total_dwelling_units,1room_flat,2room_flat,3room_flat,4room_flat,5room_flat,exec_flat,multigen_flat,studio_apartment_flat,1room_rental,2room_rental,3room_rental,other_room_rental,latitude,longitude,mall_nearest_distance,mall_within_500m,mall_within_1km,mall_within_2km,hawker_nearest_distance,hawker_within_500m,hawker_within_1km,hawker_within_2km,hawker_food_stalls,hawker_market_stalls,mrt_nearest_distance,mrt_name,bus_interchange,mrt_interchange,mrt_latitude,mrt_longitude,bus_stop_nearest_distance,bus_stop_latitude,bus_stop_longitude,pri_sch_nearest_distance,pri_sch_name,vacancy,pri_sch_affiliation,pri_sch_latitude,pri_sch_longitude,sec_sch_nearest_dist,sec_sch_name,cutoff_point,affiliation,sec_sch_latitude,sec_sch_longitude,age_sold,town_BEDOK,town_BISHAN,town_BUKIT BATOK,town_BUKIT MERAH,town_BUKIT PANJANG,town_BUKIT TIMAH,town_CENTRAL AREA,town_CHOA CHU KANG,town_CLEMENTI,town_GEYLANG,town_HOUGANG,town_JURONG EAST,town_JURONG WEST,town_KALLANG/WHAMPOA,town_MARINE PARADE,town_PASIR RIS,town_PUNGGOL,town_QUEENSTOWN,town_SEMBAWANG,town_SENGKANG,town_SERANGOON,town_TAMPINES,town_TOA PAYOH,town_WOODLANDS,town_YISHUN
0,88471,2016-05-01,KALLANG/WHAMPOA,4 ROOM,Model A,2006,680000.0,2016,5,10,12,11,4 ROOM Model A,968.760,701.928238,25,2005,Y,N,N,N,N,142,0,0,0,96,46,0,0,0,0,0,0,0,1.314299,103.872828,1094.090418,0.0,0.0,7.0,154.753357,1.0,3.0,13.0,84,60,330.083069,Kallang,0,0,1.311540,103.871731,29.427395,1.314433,103.872600,1138.633422,Geylang Methodist School,78,1,1.317659,103.882504,1138.633422,Geylang Methodist School,224,0,1.317659,103.882504,10,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False
1,122598,2012-07-01,BISHAN,5 ROOM,Improved,1987,665000.0,2012,7,7,9,8,5 ROOM Improved,1399.320,475.230826,9,1987,Y,N,N,N,N,112,0,0,0,56,56,0,0,0,0,0,0,0,1.346086,103.855078,866.9414477,0.0,1.0,3.0,640.151925,0.0,1.0,7.0,80,77,903.659703,Bishan,1,1,1.350580,103.848305,58.207761,1.345659,103.855381,415.607357,Kuo Chuan Presbyterian Primary School,45,1,1.349783,103.854529,447.894399,Kuo Chuan Presbyterian Secondary School,232,0,1.350110,103.854892,25,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,170897,2013-07-01,BUKIT BATOK,EXECUTIVE,Apartment,1997,838000.0,2013,7,13,15,14,EXECUTIVE Apartment,1550.016,540.639580,16,1996,Y,N,N,N,N,90,0,0,0,0,30,60,0,0,0,0,0,0,1.343867,103.760535,1459.579948,0.0,0.0,4.0,1762.082341,0.0,0.0,1.0,84,95,1334.251197,Bukit Batok,1,0,1.349561,103.749970,214.747860,1.344064,103.758613,498.849039,Keming Primary School,39,0,1.345245,103.756265,180.074558,Yusof Ishak Secondary School,188,0,1.342334,103.760013,16,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,86070,2012-04-01,BISHAN,4 ROOM,Model A,1992,550000.0,2012,4,1,5,3,4 ROOM Model A,1108.692,496.080066,11,1990,Y,Y,N,N,N,75,0,0,0,68,7,0,0,0,0,0,0,0,1.358245,103.845504,950.1751993,0.0,1.0,4.0,726.215262,0.0,1.0,9.0,32,86,907.453484,Bishan,1,1,1.350580,103.848305,43.396521,1.358045,103.845169,389.515528,Catholic High School,20,1,1.354789,103.844934,389.515528,Catholic High School,253,1,1.354789,103.844934,20,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,153632,2017-12-01,YISHUN,4 ROOM,Simplified,1987,298000.0,2017,12,1,3,2,4 ROOM Simplified,893.412,333.552717,4,1987,Y,N,N,N,N,48,0,0,0,30,18,0,0,0,0,0,0,0,1.414745,103.835532,729.7718948,0.0,1.0,2.0,1540.151439,0.0,0.0,1.0,45,0,412.343032,Khatib,0,0,1.4

In [5]:
feature_cols = ['floor_area_sqft','age_sold','mid','hawker_within_2km','mrt_nearest_distance',
                'town_BEDOK', 'town_BISHAN', 'town_BUKIT BATOK',
                'town_BUKIT MERAH', 'town_BUKIT PANJANG', 'town_BUKIT TIMAH',
                'town_CENTRAL AREA', 'town_CHOA CHU KANG', 'town_CLEMENTI',
                'town_GEYLANG', 'town_HOUGANG', 'town_JURONG EAST', 'town_JURONG WEST',
                'town_KALLANG/WHAMPOA', 'town_MARINE PARADE', 'town_PASIR RIS',
                'town_PUNGGOL', 'town_QUEENSTOWN', 'town_SEMBAWANG', 'town_SENGKANG',
                'town_SERANGOON', 'town_TAMPINES', 'town_TOA PAYOH', 'town_WOODLANDS',
                'town_YISHUN']

Assigning features we plan to use for our model to 'feature_cols' variable.

In [6]:
def train_test_rmse(dataframe, feature_cols):
    # Assign features and resale price from training data to X and y, respectively
    X = dataframe[feature_cols]
    y = dataframe.resale_price

    X_train, X_test, y_train, y_test = train_test_split(X, y,random_state=99)

    # Train our model using training data
    linreg = LinearRegression()
    linreg.fit(X_train, y_train)

    # Calculate predictions using our model
    y_pred = linreg.predict(X_test)
    return X_train, X_test, y_train, y_test, y_pred

X_train, X_test, y_train, y_test, y_pred = train_test_rmse(df,feature_cols)

print (f'RMSE score: {metrics.root_mean_squared_error(y_test, y_pred):.0f}, R2 score: {metrics.r2_score(y_test, y_pred):.3f}')

RMSE score: 54852, R2 score: 0.853


Created a function to train our model with different combinations of features and use the RMSE and R2 scores to optimize our model.

In [7]:
# Calculating null RMSE as the baseline model
y_null = np.zeros_like(y_test, dtype=float)
y_null.fill(y_train.mean())

print(f'Null RMSE: {metrics.root_mean_squared_error(y_test,y_null):.0f}')

Null RMSE: 142924


Our initial model has a much lower RMSE score, 54852, than the null RMSE, 142924.

In [8]:
# Testing out by removing towns with lower correlation to resale_price
feature_cols_1 = ['floor_area_sqft','age_sold','mid','hawker_within_2km','mrt_nearest_distance',
                'town_BISHAN', 'town_BUKIT BATOK',
                'town_BUKIT MERAH', 'town_BUKIT TIMAH','town_QUEENSTOWN',
                'town_CENTRAL AREA', 'town_JURONG WEST','town_PASIR RIS', 'town_WOODLANDS','town_YISHUN']

X_train, X_test, y_train, y_test, y_pred = train_test_rmse(df,feature_cols_1)

print (f'RMSE score: {metrics.root_mean_squared_error(y_test, y_pred):.0f}, R2 score: {metrics.r2_score(y_test, y_pred):.3f}')

RMSE score: 64225, R2 score: 0.798


In [9]:
# Added back removed towns and changed hawker_within_2km to hawker_nearest_distance
feature_cols_2 = ['floor_area_sqft','age_sold','mid','hawker_nearest_distance','mrt_nearest_distance',
                'town_BEDOK', 'town_BISHAN', 'town_BUKIT BATOK',
                'town_BUKIT MERAH', 'town_BUKIT PANJANG', 'town_BUKIT TIMAH',
                'town_CENTRAL AREA', 'town_CHOA CHU KANG', 'town_CLEMENTI',
                'town_GEYLANG', 'town_HOUGANG', 'town_JURONG EAST', 'town_JURONG WEST',
                'town_KALLANG/WHAMPOA', 'town_MARINE PARADE', 'town_PASIR RIS',
                'town_PUNGGOL', 'town_QUEENSTOWN', 'town_SEMBAWANG', 'town_SENGKANG',
                'town_SERANGOON', 'town_TAMPINES', 'town_TOA PAYOH', 'town_WOODLANDS',
                'town_YISHUN']

X_train, X_test, y_train, y_test, y_pred = train_test_rmse(df,feature_cols_2)

print (f'RMSE score: {metrics.root_mean_squared_error(y_test, y_pred):.0f}, R2 score: {metrics.r2_score(y_test, y_pred):.3f}')

RMSE score: 54808, R2 score: 0.853


In [10]:
# Added in new numerical features vacancy and cutoff_point
feature_cols_3 = ['floor_area_sqft','age_sold','mid','hawker_nearest_distance','mrt_nearest_distance', 'vacancy', 'cutoff_point',
                'town_BEDOK', 'town_BISHAN', 'town_BUKIT BATOK',
                'town_BUKIT MERAH', 'town_BUKIT PANJANG', 'town_BUKIT TIMAH',
                'town_CENTRAL AREA', 'town_CHOA CHU KANG', 'town_CLEMENTI',
                'town_GEYLANG', 'town_HOUGANG', 'town_JURONG EAST', 'town_JURONG WEST',
                'town_KALLANG/WHAMPOA', 'town_MARINE PARADE', 'town_PASIR RIS',
                'town_PUNGGOL', 'town_QUEENSTOWN', 'town_SEMBAWANG', 'town_SENGKANG',
                'town_SERANGOON', 'town_TAMPINES', 'town_TOA PAYOH', 'town_WOODLANDS',
                'town_YISHUN']

X_train, X_test, y_train, y_test, y_pred = train_test_rmse(df,feature_cols_3)

print (f'RMSE score: {metrics.root_mean_squared_error(y_test, y_pred):.0f}, R2 score: {metrics.r2_score(y_test, y_pred):.3f}')

RMSE score: 54748, R2 score: 0.853


In [11]:
# Added in tranc_year and removed vacancy and cutoff point
feature_cols_4 = ['floor_area_sqft','age_sold','mid','hawker_nearest_distance','mrt_nearest_distance', 'tranc_year',
                'town_BEDOK', 'town_BISHAN', 'town_BUKIT BATOK',
                'town_BUKIT MERAH', 'town_BUKIT PANJANG', 'town_BUKIT TIMAH',
                'town_CENTRAL AREA', 'town_CHOA CHU KANG', 'town_CLEMENTI',
                'town_GEYLANG', 'town_HOUGANG', 'town_JURONG EAST', 'town_JURONG WEST',
                'town_KALLANG/WHAMPOA', 'town_MARINE PARADE', 'town_PASIR RIS',
                'town_PUNGGOL', 'town_QUEENSTOWN', 'town_SEMBAWANG', 'town_SENGKANG',
                'town_SERANGOON', 'town_TAMPINES', 'town_TOA PAYOH', 'town_WOODLANDS',
                'town_YISHUN']

X_train, X_test, y_train, y_test, y_pred = train_test_rmse(df,feature_cols_4)

print (f'RMSE score: {metrics.root_mean_squared_error(y_test, y_pred):.0f}, R2 score: {metrics.r2_score(y_test, y_pred):.3f}')

RMSE score: 54762, R2 score: 0.853


After trying a few different combinations, we decided to use feature_cols_4 as our combination of features to test other prediction models.

In [12]:
# Assign y_test and y_pred values to variables for plotting
results_df = pd.DataFrame({
    'Actual': y_test,
    'Predicted': y_pred
})

# Plot of predicted resale prices vs actual resale prices
fig1 = px.scatter(results_df, x='Actual', y='Predicted', opacity=0.6, width = 600,
                 labels={'Actual': 'Actual Resale Price (SGD)', 'Predicted': 'Predicted Resale Price (SGD)'})

# Add diagonal perfect prediction line
min_val = min(results_df.min())
max_val = max(results_df.max())
fig1.add_trace(go.Scatter(mode='lines',
               x=[min_val, max_val],
               y=[min_val, max_val],
               line=dict(color='red', dash='dash'),
               name='Perfect Prediction'))

fig1.update_layout(title = {'text' : 'Predicted vs Actual Values (Linear Regression)', 'x' : 0.5, 'y' : 0.95, 'xanchor' : 'center'})
fig1.update_layout(legend=dict(x=0, y=1, xanchor='left', yanchor='top', bgcolor='rgba(255, 255, 255, 0.5)'))
fig1.show()

The linear regression model was able to accurately predict resale prices up to $800k, beyond that model performance began to drop.

<h2> 2. Data Modelling using Decision Tree Regressor Model

In [13]:
from sklearn.tree import DecisionTreeRegressor

In [14]:
# Assign features and resale price from training data to X and y, respectively
X = df[feature_cols_4]
y = df.resale_price

X_train, X_test, y_train, y_test = train_test_split(X, y,random_state=99)

# Train our model using training data
regressor = DecisionTreeRegressor(random_state=99)
regressor.fit(X_train, y_train)

# Calculate predictions using our model
y_pred = regressor.predict(X_test)

print (f'RMSE score: {metrics.root_mean_squared_error(y_test, y_pred):.0f}, R2 score: {metrics.r2_score(y_test, y_pred):.3f}')

RMSE score: 35332, R2 score: 0.939


In [15]:
# Re-train the model with full X and y dataset
regressor.fit(X, y)

# Assign features and resale price from test data to X and y, respectively
X = test_df[feature_cols_4]
y = test_df.resale_price

# Calculate predictions using our model
y_pred = regressor.predict(X)

print (f'RMSE score: {metrics.root_mean_squared_error(y, y_pred):.0f}, R2 score: {metrics.r2_score(y, y_pred):.3f}')

RMSE score: 33916, R2 score: 0.944


In [16]:
# Retrieve default parameters for the Decision Tree Regressor mdoel
regressor.get_params()

{'ccp_alpha': 0.0,
 'criterion': 'squared_error',
 'max_depth': None,
 'max_features': None,
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'monotonic_cst': None,
 'random_state': 99,
 'splitter': 'best'}

Using the decision tree model which we trained with the training data, we predicted resale prices of the test data, and obtained much lower RMSE score and higher R2 score.

In [17]:
from sklearn.model_selection import GridSearchCV

In [ ]:
# Define the parameter grid to search
param_grid = {'max_depth': [3, 5, 7, None],  
              'min_samples_split': list(range(2,20)),
              'min_samples_leaf': [1, 2, 4, 8]}

grid_search = GridSearchCV(estimator=DecisionTreeRegressor(random_state=99),
                           param_grid=param_grid,
                           cv=2, 
                           scoring='neg_mean_squared_error', 
                           n_jobs=-1, 
                           verbose=1)

# Use Grid Search to find the best parameters for our model
grid_search.fit(X_train, y_train)

# Obtain optimized parameters
print("\nBest Hyperparameters found by GridSearchCV:")
print(grid_search.best_params_)

# Evaluate the best model
y_pred_tuned = grid_search.best_estimator_.predict(X_test)

print(f'RMSE score: {metrics.root_mean_squared_error(y_test, y_pred_tuned):.0f}')
print(f'R2 score: {metrics.r2_score(y_test, y_pred_tuned):.3f}')

Fitting 2 folds for each of 288 candidates, totalling 576 fits

Best Hyperparameters found by GridSearchCV:
{'max_depth': None, 'min_samples_leaf': 2, 'min_samples_split': 16}
RMSE score: 32691
R2 score: 0.948


Using a GridSearchCV function, we fine-tune the model's hyperparameters to obtain even lower RMSE and high R2 scores.

In [19]:
# Re-train the model with full X and y dataset
X = df[feature_cols_4]
y = df.resale_price
grid_search.fit(X, y)

# Assign features and resale price from test data to X and y, respectively
X = test_df[feature_cols_4]
y = test_df.resale_price

# Calculate predictions using our optimized model
y_pred_tuned = grid_search.best_estimator_.predict(X)

print(f'RMSE score: {metrics.root_mean_squared_error(y, y_pred_tuned):.0f}')
print(f'R2 score: {metrics.r2_score(y, y_pred_tuned):.3f}')

Fitting 2 folds for each of 288 candidates, totalling 576 fits
RMSE score: 31189
R2 score: 0.953


With the tuned decision tree regressor model, we obtained slightly better RMSE and R2 scores.

In [20]:
# Retrieve feature importance from decision tree model
importances = grid_search.best_estimator_.feature_importances_

# Put into a DataFrame for easy viewing
feature_importance_df = pd.DataFrame({
    'feature': feature_cols_4,
    'importance': importances
}).sort_values(by='importance', ascending=True)

feature_importance_df

,feature,importance
22,town_PUNGGOL,0.000315
15,town_GEYLANG,0.000638
8,town_BUKIT BATOK,0.001212
25,town_SENGKANG,0.001396
21,town_PASIR RIS,0.001445
17,town_JURONG EAST,0.001577
24,town_SEMBAWANG,0.001689
20,town_MARINE PARADE,0.001762
16,town_HOUGANG,0.002074
13,town_CHOA CHU KANG,0.002197


In [21]:
# Plot of features by importance in decision tree regressor model
fig2 = px.bar(feature_importance_df.tail(10),
              x='importance',
              y='feature',
              orientation='h',
              labels={'feature': 'Features', 'importance': 'Importance'},
              width = 800)
fig2.update_layout(title = {'text' : 'Feature Importance in Decision Tree Regressor', 'x' : 0.5, 'y' : 0.95, 'xanchor' : 'center', 'font': {'size': 20} }, 
                  xaxis_title_font={'size': 18}, 
                  yaxis_title_font={'size': 18},)
fig2.update_layout(yaxis = dict(
                   tickvals = ['floor_area_sqft', 'mid', 'hawker_nearest_distance', 'age_sold', 'mrt_nearest_distance', 'tranc_year', 'town_BUKIT MERAH', 'town_BISHAN', 'town_WOODLANDS', 'town_QUEENSTOWN'],
                   ticktext = ['Floor Area in sqft', 'Floor', 'Distance to Nearest Hawker', 'HDB Age when Sold', 'Distance to Nearest MRT', 'Transaction Year', 'Bukit Merah', 'Bishan', 'Woodlands', 'Queenstown']))
fig2.show()

The plot of feature importance tells us that floor_area_sqft, mid (floor), hawker_nearest_distance, age_sold, and mrt_nearest_distance are the top 5 most important features in this model. <br>

This features differ slightly from the correlation matrix. For example, hawker_nearest_distance displayed very low r values on the matrix but was ranked high for the decision tree model. <br>

This is because the decision tree regressor produces predictions on a rule-based approach and is able to handle non-linearity and complex interactions very well, hence capturing the non-linear relationship between resale price and both hawker_nearest_distance.

In [22]:
# Assign y_test and y_pred values to variables for plotting
results_df = pd.DataFrame({
    'Actual': y,
    'Predicted': y_pred_tuned
})

# Plot of predicted resale prices vs actual resale prices
fig3 = px.scatter(results_df, x='Actual', y='Predicted', opacity=0.6, width = 600,
                 labels={'Actual': 'Actual Resale Price (SGD)', 'Predicted': 'Predicted Resale Price (SGD)'})

# Add diagonal perfect prediction line
min_val = min(results_df.min())
max_val = max(results_df.max())
fig3.add_trace(go.Scatter(mode='lines',
               x=[min_val, max_val],
               y=[min_val, max_val],
               line=dict(color='red', dash='dash'),
               name='Perfect Prediction'))

fig3.update_layout(title = {'text' : 'Predicted vs Actual Values (Decision Tree Regressor)', 'x' : 0.5, 'y' : 0.95, 'xanchor' : 'center'})
fig3.update_layout(legend=dict(x=0, y=1, xanchor='left', yanchor='top', bgcolor='rgba(255, 255, 255, 0.5)'))
fig3.show()

The decision tree regressor displayed high performance in predicting resale prices for the entire range of resale prices. Significant improvement in model performance compared to the linear regression model.

<h2> 3. Data Modelling using Random Forest Regressor Model

In [23]:
from sklearn.ensemble import RandomForestRegressor

In [24]:
# Assign features and resale price from training data to X and y, respectively
X = df[feature_cols_4]
y = df.resale_price

X_train, X_test, y_train, y_test = train_test_split(X, y,random_state=99)

# Train our model using training data
rf_regressor = RandomForestRegressor(random_state=99)
rf_regressor.fit(X_train, y_train)

# Calculate predictions using our model
y_pred = rf_regressor.predict(X_test)

print (f'RMSE score: {metrics.root_mean_squared_error(y_test, y_pred):.0f}, R2 score: {metrics.r2_score(y_test, y_pred):.3f}')

RMSE score: 26157, R2 score: 0.967


In [25]:
# Retrieve default parameters for the Random Forest Regressor mdoel
rf_regressor.get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'criterion': 'squared_error',
 'max_depth': None,
 'max_features': 1.0,
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'monotonic_cst': None,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': 99,
 'verbose': 0,
 'warm_start': False}

In [26]:
# Re-train the model with full X and y dataset
rf_regressor.fit(X, y)

# Assign features and resale price from test data to X and y, respectively
X = test_df[feature_cols_4]
y = test_df.resale_price

# Calculate predictions using our model
y_pred = rf_regressor.predict(X)

print (f'RMSE score: {metrics.root_mean_squared_error(y, y_pred):.0f}, R2 score: {metrics.r2_score(y, y_pred):.3f}')

RMSE score: 25818, R2 score: 0.968


<b>Random Forest Regressor is our chosen model</b> because it produced the lowest RMSE and highest R2 score when used to predict the resale prices of the testing data. 

In [27]:
# Get feature importances
importances = rf_regressor.feature_importances_

# Put into a DataFrame for easy viewing
feature_importance_df = pd.DataFrame({
    'feature': feature_cols_4,
    'importance': importances
}).sort_values(by='importance', ascending=True)

feature_importance_df

,feature,importance
22,town_PUNGGOL,0.000508
15,town_GEYLANG,0.000875
8,town_BUKIT BATOK,0.001210
25,town_SENGKANG,0.001275
21,town_PASIR RIS,0.001332
17,town_JURONG EAST,0.001632
24,town_SEMBAWANG,0.001636
16,town_HOUGANG,0.001909
13,town_CHOA CHU KANG,0.002151
28,town_TOA PAYOH,0.003010


In [28]:
# Plot of features by importance in random forest regressor model
fig4 = px.bar(feature_importance_df.tail(10),
              x='importance',
              y='feature',
              orientation='h',
              labels={'feature': 'Features', 'importance': 'Importance'},
              width = 800)
fig4.update_layout(title = {'text' : 'Feature Importance in Random Forest Regressor', 'x' : 0.5, 'y' : 0.95, 'xanchor' : 'center', 'font': {'size': 20} }, 
                  xaxis_title_font={'size': 18}, 
                  yaxis_title_font={'size': 18},)
fig4.update_layout(yaxis = dict(
                   tickvals = ['floor_area_sqft', 'mid', 'hawker_nearest_distance', 'age_sold', 'mrt_nearest_distance', 'tranc_year', 'town_BUKIT MERAH', 'town_BISHAN', 'town_WOODLANDS', 'town_QUEENSTOWN'],
                   ticktext = ['Floor Area in sqft', 'Floor', 'Distance to Nearest Hawker', 'HDB Age when Sold', 'Distance to Nearest MRT', 'Transaction Year', 'Bukit Merah', 'Bishan', 'Woodlands', 'Queenstown']))
fig4.show()

Similar to the decision tree model, the random forest model produces predictions on a rule-based approach and is able to handle non-linearity and complex interactions very well, hence capturing the non-linear relationship between resale price and hawker_nearest_distance.

In [29]:
# Assign y_test and y_pred values to variables for plotting
results_df = pd.DataFrame({
    'Actual': y,
    'Predicted': y_pred
})

# Plot of predicted resale prices vs actual resale prices
fig5 = px.scatter(results_df, x='Actual', y='Predicted', opacity=0.6, width = 600,
                 labels={'Actual': 'Actual Resale Price (SGD)', 'Predicted': 'Predicted Resale Price (SGD)'})

# Add diagonal perfect prediction line
min_val = min(results_df.min())
max_val = max(results_df.max())
fig5.add_trace(go.Scatter(mode='lines',
               x=[min_val, max_val],
               y=[min_val, max_val],
               line=dict(color='red', dash='dash'),
               name='Perfect Prediction'))

fig5.update_layout(title = {'text' : 'Predicted vs Actual Values (Random Forest Regressor)', 'x' : 0.5, 'y' : 0.95, 'xanchor' : 'center'})
fig5.update_layout(legend=dict(x=0, y=1, xanchor='left', yanchor='top', bgcolor='rgba(255, 255, 255, 0.5)'))
fig5.show()

The random forest regressor displayed high performance in predicting resale prices for the entire range of resale prices and also a lower deviation from the perfect prediction line as compared to the decision tree regressor model. Improvement in model performance compared to the decision tree regressor model.

In [ ]:
# Assign features and resale price from training data to X and y, respectively
X = df[feature_cols_4]
y = df.resale_price

X_train, X_test, y_train, y_test = train_test_split(X, y,random_state=99)

# Define the parameter grid to search
param_grid = {
    'n_estimators': [100],
    'max_depth': [3, 5, 7, None], 
    'min_samples_split': list(range(2,20)),
    'min_samples_leaf': [1, 2, 4, 8]
}

grid_search = GridSearchCV(estimator=RandomForestRegressor(random_state=99),
                           param_grid=param_grid,
                           cv=2, 
                           scoring='neg_mean_squared_error', 
                           n_jobs=-1, 
                           verbose=1)

# Use Grid Search to find the best parameters for our model
grid_search.fit(X_train, y_train)

# Obtain optimized parameters
print("\nBest Hyperparameters found by GridSearchCV:")
print(grid_search.best_params_)

# Evaluate the best model
y_pred_tuned = grid_search.best_estimator_.predict(X_test)

print(metrics.root_mean_squared_error(y_test, y_pred_tuned))
print(metrics.r2_score(y_test, y_pred_tuned))

In [ ]:
# Re-train the model with full X and y dataset
X = df[feature_cols_4]
y = df.resale_price
grid_search.fit(X, y)

# # Evaluate the best model
X = test_df[feature_cols_4]
y = test_df.resale_price

y_pred_tuned = grid_search.best_estimator_.predict(X)

print(f'RMSE score: {metrics.root_mean_squared_error(y, y_pred_tuned):.0f}')
print(f'R2 score: {metrics.r2_score(y, y_pred_tuned):.3f}')

For our random forest regressor model, hypertuning parameters makes almost no difference to the RMSE and R2 score.

In [ ]:
kaggle_submission = test_df['id']
kaggle_submission = pd.DataFrame(kaggle_submission)
kaggle_submission['Predicted'] = y_pred_tuned
kaggle_submission.to_csv('./Model Exported files/kaggle_submission.csv', index = False)

Created a dataframe containing the id and predicted resale prices using our model and exported as a csv file.